# Create Boundary Conditions of the 79NG Fjord GETM Setup

This notebook creates the GETM input files describing the boundary conditions of the 79NG fjord setup.

Notebook by Markus Reinert (IOW, 2023, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

## Load the topography

In [ ]:
topo = xr.open_dataset("model/topography_1km.nc")
topo

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6), dpi=300)
ax.pcolormesh(topo.mask)
ax.set_title("GETM index 1 is between grid lines 0 and 1\nGETM index 11 is above/to the right of grid line 10")
ax.set_aspect("equal")
ax.set_xticks(np.arange(0, topo.lon.size + 1, 10))
ax.set_yticks(np.arange(0, topo.lat.size + 1, 10))
ax.set_xticks(np.arange(topo.lon.size + 1), minor=True)
ax.set_yticks(np.arange(topo.lat.size + 1), minor=True)
ax.grid(which="minor", linewidth=0.5)
ax.grid(color="red", linewidth=0.5)

## Create boundary info file

The file format is explained at https://getm.eu/bdys/articles/bdys.html.

In [ ]:
boundary_info = f"""\
# no western boundary
0
# northern boundary
1
{topo.lat.size} {np.where(topo.mask.isel(lat=-1))[0][0] + 1} {topo.lon.size - 1} 3 0
# eastern boundary over the full model domain
1
{topo.lon.size} 1 {topo.lat.size} 3 0
# southern boundaries
2
"""
# Get start index of the first boundary section
i_boundary_start = np.where(topo.mask.isel(lat=0))[0][0]
# Get end index of the first boundary section plus one (p1)
i_boundary_endp1 = np.where(~topo.mask.isel(lat=0, lon=slice(i_boundary_start, None)))[0][0] + i_boundary_start
boundary_info += f"1 {i_boundary_start + 1} {i_boundary_endp1} 3 0\n"
# Get start index of the second boundary section
i_boundary_start = np.where(topo.mask.isel(lat=0, lon=slice(i_boundary_endp1, None)))[0][0] + i_boundary_endp1
boundary_info += f"1 {i_boundary_start + 1} {topo.lon.size - 1} 3 0\n"

In [ ]:
bdyinfo_filename = "model/bdyinfo.dat"
with open(bdyinfo_filename, "w") as f:
    f.write(boundary_info)
print(f"Saved the following boundary info as {bdyinfo_filename!r}:\n{boundary_info}")